# <b>Task:</b> Implementation and comparison of algorithms for solving quadratic assignment problem

The data for the work is attached in the repository. 

Structure:

n - number of plants and places

next n lines - distance matrix D empty line 

next n lines - flow matrix F

## Import libraries, reading data, support functions

In [ ]:
import random
import numpy as np
import time
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/qap_test.zip'


Archive:  /content/drive/MyDrive/qap_test.zip
  inflating: tai100a                 
   creating: __MACOSX/
  inflating: __MACOSX/._tai100a      
  inflating: tai20a                  
  inflating: __MACOSX/._tai20a       
  inflating: tai40a                  
  inflating: __MACOSX/._tai40a       
  inflating: tai60a                  
  inflating: __MACOSX/._tai60a       
  inflating: tai80a                  
  inflating: __MACOSX/._tai80a       


In [ ]:
def check_time(f, *args):
  function_time = 0.
  for i in np.arange(10):  #at 1,000 and 100, it takes a long time.
    start = time.time()
    f(*args)
    function_time += time.time() - start
  avarege_time = function_time / 10
  return avarege_time

In [ ]:
def read_data_for_test(file_path):
  matrix_d = []
  with open(file_path) as f:
    size = int(f.readline())
    for i in range(size):
      mat = f.readline()
      matrix_d.append(list(map(int, mat.split())))
    empty = f.readline()
    matrix_f = [list(map(int, row.split())) for row in f.readlines()]
    return size, matrix_d, matrix_f

In [ ]:
def print_data_for_test(size, matrix_d, matrix_f):
  print("ДАННЫЕ:\n")
  print("Количество заводов и мест: ", size)
  print("\n====================================================== Матрица расстояний D ======================================================\n\n")
  for i in range(size):
    for j in range(size):
      print(matrix_d[i][j],end=" ")
    print('\n')
  print("\n====================================================== Mатрица потока F ==========================================================\n\n")
  for k in range(size):
    for l in range(size):
      print(matrix_f[k][l],end=" ")
    print('\n')




In [ ]:
 #reading test
size, matrix_d, matrix_f = read_data_for_test('tai20a')
print_data_for_test(size, matrix_d, matrix_f)
size, matrix_d, matrix_f = read_data_for_test('tai40a')
print_data_for_test(size, matrix_d, matrix_f)
size, matrix_d, matrix_f = read_data_for_test('tai60a')
print_data_for_test(size, matrix_d, matrix_f)
size, matrix_d, matrix_f = read_data_for_test('tai80a')
print_data_for_test(size, matrix_d, matrix_f)
size, matrix_d, matrix_f = read_data_for_test('tai100a')
print_data_for_test(size, matrix_d, matrix_f)


ДАННЫЕ:

Количество заводов и мест:  20

====================================================== Матрица расстояний D ======================================================


0 85 72 7 49 46 87 58 17 68 27 21 6 67 26 82 44 35 3 62 

85 0 8 51 1 91 39 87 72 45 96 7 87 68 33 3 21 90 45 47 

72 8 0 25 30 43 97 33 35 61 42 36 43 7 84 6 0 0 48 62 

7 51 25 0 59 29 94 82 29 3 3 51 67 39 15 66 42 23 62 62 

49 1 30 59 0 28 76 66 82 98 35 15 17 77 44 26 76 86 60 62 

46 91 43 29 28 0 62 83 91 57 62 36 2 2 43 65 37 49 61 5 

87 39 97 94 76 62 0 34 53 96 82 48 28 31 75 1 95 7 92 69 

58 87 33 82 66 83 34 0 62 32 97 5 39 50 82 93 71 35 14 20 

17 72 35 29 82 91 53 62 0 74 49 50 37 79 19 51 70 42 26 79 

68 45 61 3 98 57 96 32 74 0 98 60 35 9 96 70 21 37 37 67 

27 96 42 3 35 62 82 97 49 98 0 93 93 39 2 52 26 90 26 1 

21 7 36 51 15 36 48 5 50 60 93 0 68 93 7 94 19 54 37 0 

6 87 43 67 17 2 28 39 37 35 93 68 0 20 12 11 66 84 80 1 

67 68 7 39 77 2 31 50 79 9 39 93 20 0 55 9 21 12 65 7 

26 33 84 15

## Local search + first-improvement + don't look bits

In [ ]:
def objective_function_value(size, matrix_d, matrix_f, sol):
    value = 0
    for i in range(size):
        for j in range(size):
            value += matrix_d[i][j] * matrix_f[sol[i]][sol[j]]
    return value

def change_val(size,matrix_d, matrix_f, sol, i, j, val):
    for x in range(size):
        val -= matrix_d[i][x] * matrix_f[sol[i]][sol[x]]
        val -= matrix_d[j][x] * matrix_f[sol[j]][sol[x]]
    sol[i], sol[j] = sol[j], sol[i]
    for x in range(size):
        val += matrix_d[i][x] * matrix_f[sol[i]][sol[x]]
        val += matrix_d[j][x] * matrix_f[sol[j]][sol[x]]
    return sol, val

def initial_solution(size):
  sol = list(np.arange(size))
  random.shuffle(sol)
  return sol


In [ ]:
def local_search(size, matrix_d, matrix_f, base):
    sol = base.copy()
    val = objective_function_value(size, matrix_d, matrix_f, sol)

    dont_look = np.zeros(size)
    i = 0
    for i in range(size):
      if dont_look[i] == 1:
            i += 1
            continue
      flag = False
      for j in range(i + 1, size):
        new_sol, new_val = change_val(size, matrix_d, matrix_f, sol.copy(), i, j, val)
        if new_val < val:
          sol = new_sol
          val = new_val
          flag = True
          i = 0
          break
        if not flag:
            dont_look[i] = 1
           # print(dont_look)
    return sol, val

def local_search_first_dontlook(size,matrix_d, matrix_f):
    return local_search(size,matrix_d,matrix_f,initial_solution(size))

In [ ]:
files_name = ['tai20a','tai40a','tai60a','tai80a','tai100a']
for file_name in files_name:
  size, matrix_d, matrix_f = read_data_for_test(file_name)
  answer,val = local_search_first_dontlook(size,matrix_d, matrix_f)
  print("Тест: ", file_name )
  print("Ответ: ", answer)
  print("Значение: ", val)
  print("Время работы: ", check_time(local_search_first_dontlook, size,matrix_d, matrix_f), "\n")


Тест:  tai20a
Ответ:  [12, 1, 11, 2, 16, 19, 15, 3, 0, 10, 14, 4, 5, 9, 13, 7, 18, 6, 8, 17]
Значение:  877621
Время работы:  0.0015784740447998048 

Тест:  tai40a
Ответ:  [6, 24, 7, 36, 22, 26, 1, 18, 10, 4, 21, 25, 38, 17, 15, 11, 5, 2, 14, 20, 32, 39, 9, 35, 31, 3, 34, 19, 8, 27, 12, 16, 33, 30, 28, 37, 0, 23, 13, 29]
Значение:  3626126
Время работы:  0.008538126945495605 

Тест:  tai60a
Ответ:  [53, 50, 19, 3, 4, 7, 5, 51, 52, 36, 2, 30, 1, 47, 27, 12, 34, 20, 35, 54, 10, 43, 28, 17, 18, 45, 31, 55, 8, 41, 15, 25, 56, 22, 37, 58, 9, 40, 13, 42, 49, 24, 16, 39, 44, 21, 57, 26, 32, 38, 11, 46, 48, 33, 29, 23, 59, 0, 6, 14]
Значение:  8183467
Время работы:  0.02270381450653076 

Тест:  tai80a
Ответ:  [38, 68, 9, 79, 40, 77, 56, 62, 3, 53, 61, 69, 63, 24, 65, 15, 36, 46, 35, 54, 42, 49, 45, 10, 43, 4, 22, 37, 66, 57, 51, 12, 47, 72, 64, 1, 26, 20, 41, 25, 31, 5, 18, 44, 50, 8, 17, 39, 59, 70, 71, 7, 0, 11, 74, 78, 29, 52, 73, 16, 2, 30, 55, 19, 21, 14, 27, 32, 76, 58, 33, 6, 48, 13, 23

## Iterated local search

In [ ]:
def stochastic_two_opt(sol, i, j):
    if i == 0:
        new_sol = sol[:i] + sol[j::-1] + sol[j+1:]
    else:
        new_sol = sol[:i] + sol[j:i-1:-1] + sol[j+1:]
    return new_sol

In [ ]:
def iterated_local_search(size, matrix_d, matrix_f):
    sol, val = local_search(size,matrix_d, matrix_f, initial_solution(size))
    count = 0
    k = size // 2
    i = random.randint(0, size-2)
    j = max(i + random.randint(1, k), size-1)
    while count < 100:
        count += 1
        tmp_sol = stochastic_two_opt(sol, i, j)
        new_sol, new_val = local_search(size, matrix_d, matrix_f, tmp_sol)
        if new_val < val:
            val = new_val
            sol = new_sol
            count = 0
    return sol, val

In [ ]:
files_name = ['tai20a','tai40a','tai60a','tai80a','tai100a']
for file_name in files_name:
  size, matrix_d, matrix_f = read_data_for_test(file_name)
  answer,val = iterated_local_search(size,matrix_d, matrix_f)
  print("Тест: ", file_name )
  print("Ответ: ", answer)
  print("Значение: ", val)
  print("Время работы: ", check_time(iterated_local_search, size,matrix_d, matrix_f), "\n")


Тест:  tai20a
Ответ:  [12, 11, 4, 14, 19, 9, 2, 7, 18, 8, 13, 15, 6, 17, 10, 3, 1, 0, 16, 5]
Значение:  791472
Время работы:  0.1756258487701416 

Тест:  tai40a
Ответ:  [5, 35, 38, 20, 13, 24, 27, 12, 39, 8, 10, 19, 18, 4, 1, 34, 9, 29, 23, 15, 31, 11, 0, 21, 32, 2, 25, 33, 37, 3, 30, 26, 7, 28, 36, 6, 16, 14, 22, 17]
Значение:  3452312
Время работы:  1.1954882144927979 

Тест:  tai60a
Ответ:  [55, 5, 27, 11, 34, 50, 57, 19, 15, 40, 49, 54, 53, 43, 25, 8, 29, 42, 23, 18, 59, 30, 10, 52, 56, 2, 51, 28, 13, 35, 47, 14, 17, 3, 36, 38, 20, 9, 0, 31, 26, 48, 22, 1, 44, 58, 24, 32, 16, 37, 4, 6, 39, 41, 21, 46, 7, 33, 12, 45]
Значение:  8116532
Время работы:  3.5146037578582763 

Тест:  tai80a
Ответ:  [75, 52, 63, 65, 79, 70, 23, 50, 15, 36, 55, 53, 38, 29, 16, 66, 28, 20, 73, 62, 11, 68, 5, 64, 32, 31, 27, 14, 6, 45, 19, 30, 8, 44, 3, 37, 12, 9, 22, 54, 48, 77, 39, 40, 7, 51, 46, 74, 24, 67, 26, 42, 61, 1, 13, 25, 0, 47, 10, 57, 41, 35, 71, 76, 2, 33, 72, 4, 18, 49, 43, 56, 78, 21, 69, 59, 